In [147]:
# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import CharacterTextSplitter

# # 加载 PDF 数据
# loader = PyPDFLoader("./111.text")  
# documents = loader.load()

# # 拆分文档
# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20)
# texts = text_splitter.split_documents(documents)

# # 打印结果
# print(texts)

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Load text data
loader = TextLoader("./111.text")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
print(texts)


[Document(metadata={'source': './111.text'}, page_content="Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.\nMichael is also an employee at prismaticAI, where he works as a data scientist. He joined the company two years ago after completing his graduate studies.\nprismaticAI is a well-known technology company that specializes in developing cutting-edge software solutions and artificial intelligence applications. The company has a diverse workforce of talented individuals from various backgrounds.\nBoth Sarah and Michael are highly skilled professionals who contribute significantly to prismaticAI's success. They work closely with their respective teams to develop innovative products and services that meet the evolving needs of the company's clients.")]


In [148]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="llama3.1:8b",temperature=0)
# 提取知识图
llm_transformer = LLMGraphTransformer(llm=model)
# 将文本转换为图文档 
graph_documents = llm_transformer.convert_to_graph_documents(texts)
print(f"Graph Documents:{graph_documents}")
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Graph Documents:[GraphDocument(nodes=[Node(id='Sarah', type='Person', properties={}), Node(id='Michael', type='Person', properties={}), Node(id='prismaticAI', type='Company', properties={})], relationships=[Relationship(source=Node(id='prismaticAI', type='Company', properties={}), target=Node(id='Sarah', type='Person', properties={}), type='EMPLOYED_BY', properties={}), Relationship(source=Node(id='prismaticAI', type='Company', properties={}), target=Node(id='Michael', type='Person', properties={}), type='EMPLOYED_BY', properties={}), Relationship(source=Node(id='Sarah', type='Person', properties={}), target=Node(id='prismaticAI', type='Company', properties={}), type='WORKS_AT', properties={}), Relationship(source=Node(id='Michael', type='Person', properties={}), target=Node(id='prismaticAI', type='Company', properties={}), type='WORKS_AT', properties={}), Relationship(source=Node(id='Sarah', type='Person', properties={}), target=Node(id='Michael', type='Person', properties={}), type='

In [149]:
# 将知识图谱存储在数据库中
import os
from langchain_community.graphs import Neo4jGraph
# Store Knowledge Graph in Neo4j
# os.environ["NEO4J_URI"] = "neo4j+s://40b35635.databases.neo4j.io"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] = "y1dLeH4KFCUkcNhDTyv6Dpo0Q-Aqd_v_p-a0NqjswzM"
# graph = Neo4jGraph()
# graph.query("helo")
graph = Neo4jGraph(url="neo4j+s://40b35635.databases.neo4j.io", username="neo4j", password="y1dLeH4KFCUkcNhDTyv6Dpo0Q-Aqd_v_p-a0NqjswzM")
graph.add_graph_documents(graph_documents)

# 更新图模式
graph.refresh_schema()

# 查询所有节点和关系
all_nodes_and_relationships = graph.query("MATCH (n)-[r]->(m) RETURN n, r, m")
for record in all_nodes_and_relationships:
    print(record)


{'n': {'id': 'Sarah'}, 'r': ({'id': 'Sarah'}, 'WORKS_AT', {'id': 'prismaticAI'}), 'm': {'id': 'prismaticAI'}}
{'n': {'id': 'Sarah'}, 'r': ({'id': 'Sarah'}, 'COLLABORATES_WITH', {'id': 'Michael'}), 'm': {'id': 'Michael'}}
{'n': {'id': 'Michael'}, 'r': ({'id': 'Michael'}, 'WORKS_AT', {'id': 'prismaticAI'}), 'm': {'id': 'prismaticAI'}}
{'n': {'id': 'prismaticAI'}, 'r': ({'id': 'prismaticAI'}, 'EMPLOYED_BY', {'id': 'Sarah'}), 'm': {'id': 'Sarah'}}
{'n': {'id': 'prismaticAI'}, 'r': ({'id': 'prismaticAI'}, 'EMPLOYED_BY', {'id': 'Michael'}), 'm': {'id': 'Michael'}}
{'n': {'id': 'prismaticAI'}, 'r': ({'id': 'prismaticAI'}, 'HAS_EMPLOYEE', {'id': 'Sarah'}), 'm': {'id': 'Sarah'}}
{'n': {'id': 'prismaticAI'}, 'r': ({'id': 'prismaticAI'}, 'HAS_EMPLOYEE', {'id': 'Michael'}), 'm': {'id': 'Michael'}}


In [174]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore


storage_context = StorageContext.from_defaults(graph_store=graph)
print(storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x7fa8e877b990>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x7fa8e877be50>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fa8e99e1290>, property_graph_store=None)


In [214]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
# llm = ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    api_key="sk-dUaahPQNqNtDfdFgB4C7C6C711A94fF29bAa8c0d93Bd09E8",
    temperature=0,
    base_url="https://api.gptapi.us/v1/chat/completions"
)

# llm = Ollama
# llm = OllamaLLM(model="llama3.2:3b", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True, allow_dangerous_requests=True)
response = chain.invoke({"query": "Does Michael work for the same company as Sarah?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Person {id: "Michael"})-[:WORKS_AT]->(c:Company)<-[:WORKS_AT]-(s:Person {id: "Sarah"})
RETURN COUNT(c) > 0 AS worksForSameCompany

Full Context:
[{'worksForSameCompany': True}]

> Finished chain.


{'query': 'Does Michael work for the same company as Sarah?',
 'result': 'Yes, Michael works for the same company as Sarah.'}